In [2]:

## imports
import numpy as np



In [3]:

# integer multiplication
def intMM(a, b):
    
    base = 2 ** 8
    
    # turn into an array with a given base
    def getX(num):
        X = []
        d = num
        
        # while there are more digits
        while d > 0:
            # this would be unneccesary in C, since the number would already be stored
            #   in little endian order
            d, m = divmod(d, base)
            X.append(m)
        
        X.append(0)
        return X
    
    # get back an integer from the base
    def fromX(X):
        num = 0
        bi = 1
        
        # O(n * 1) (also would need to carry)
        for x in X:
            # we use multiplication by a base here; this is O(1) in C though;
            #  because it is a fixed size
            # round to nearest for floating point
            num += bi * int(x + 0.5)
            bi *= base
        
        return num
        
    # arrays
    aX = getX(a)
    bX = getX(b)
    
    
    # add 0s and pad it
    if len(aX) > len(bX):
        bX = bX + [0] * (len(aX) - len(bX))
    elif len(aX) < len(bX):
        aX = aX + [0] * (len(bX) - len(aX))
        
    assert len(aX) == len(bX)
    
    # zero pad
    aX += [0] * len(aX)
    bX += [0] * len(bX)

    # frequencies of digits
    faX = np.fft.fft(aX)
    fbX = np.fft.fft(bX)
    
    # pointwise multiplication of frequencies, which convolves the integers
    # think about it:
    #
    #   34
    # x 12
    # ----
    #   68
    #  340
    # ----
    #  408
    #
    # Each of the rows in the second sum is the original sequence (34)
    #   offset by the index of the digit in the second sequence (12),
    #   and scaled by the magnitude
    # This is:
    # sum(sum(A[j] * B[i] * 10**i for j in range(n)) for i in range(n))
    #
    # And so, any particular digit (digit 'd') of the output is:
    # C[d] = sum(A[i - d] * B[i + d] for i in range(n))
    # (not accounting for carry)
    #
    # So, this is a linear sequence that can be modeled as an IR which
    #   the 2 sequences are convolving in
    #
    # Therefore, their product is the convolution of the digits (and carry must be
    #   executed)
    #
    # Since convolve(A, B) == iFFT(FFT(A) .* FFT(B))
    # We can compute it quicker via O(n * logn) FFT algorithm + O(n) multiplication algorithm
    #
    fcX = faX * fbX
    
    # compute inverse FFT
    cX = np.real(np.fft.ifft(fcX))
    
    #print (aX, bX, cX)
    
    # return from bases
    return fromX(cX)

print (intMM(1234, 5), 1234 * 5)

#print (intMM(1000, 2))

for i in (0, 1, 500, 2320, 19234238):
    for j in (500, 239, 5938283912):
        print()
        print (i, j)
        print ((i * j), intMM(i, j))
# now, do it via qiscit

    

6170 6170

0 500
0 0

0 239
0 0

0 5938283912
0 0

1 500
500 500

1 239
239 239

1 5938283912
5938283912 5938283912

500 500
250000 250000

500 239
119500 119500

500 5938283912
2969141956000 2969141956000

2320 500
1160000 1160000

2320 239
554480 554480

2320 5938283912
13776818675840 13776818675840

19234238 500
9617119000 9617119000

19234238 239
4596982882 4596982882

19234238 5938283912
114218366074979056 114218366074979056
